# Klasifikasi Teks menggunakan Fine Tuning BERT

## Instalasi dan *import library*

In [1]:
!pip install tensorflow
!pip install  transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 31.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
     |████████████████████████████████| 120 kB 66.1 MB/s 


In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow_hub as hub
from transformers import TFAutoModel, BertTokenizerFast

from gensim.utils import simple_preprocess

from sklearn.metrics import classification_report, confusion_matrix

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

## Membaca file dan *preprocessing* data
Membaca file dan melakukan pemasukan ke dalam variabel training dan testing.

In [3]:
train_csv = 'https://drive.google.com/uc?export=download&id=10SpT9n2mqoOfDs0GznlQoOgkkKUQoKf-'
test_csv = 'https://drive.google.com/uc?export=download&id=13SBoG_QLCk-4GA79n_PyUEwA42Z8RvGd'

train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

### Understanding data

In [4]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21601 entries, 0 to 21600
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  21601 non-null  int64 
 1   text_a      21601 non-null  object
 2   label       21601 non-null  object
dtypes: int64(1), object(2)
memory usage: 506.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2800 entries, 0 to 2799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  2800 non-null   object
 1   label   2800 non-null   object
dtypes: object(2)
memory usage: 43.9+ KB
None


In [5]:
print("Any null values on train data:", train.isnull().values.any())
print("Any null values on test data:", test.isnull().values.any())

Any null values on train data: False
Any null values on test data: False


In [6]:
print("Jumlah data training: ", len(train))
print("Jumlah data testng: ", len(test))

Jumlah data training:  21601
Jumlah data testng:  2800


In [7]:
train.head(5)

,Unnamed: 0,text_a,label
0,0,betewe buka twitter cuman ngetweet liat home b...,no
1,1,mas piyuuu mugo2 corona tuh mulut tersumpal ma...,no
2,2,e100ss gini buka informasi sejelas nya identit...,yes
3,3,neng solo wes ono terduga corona cobo neng ati...,no
4,4,midiahn nii akun gak takut takut nya isu coron...,no


In [8]:
test.head(5)

,text_a,label
0,jek dajal ga depok bang,no
1,detikcom untung depok masuk wilayah nya ridwan...,no
2,df dom jakarta depok yg gunain vc cabang nya c...,no
3,your2rl depok jkt,no
4,doakan indonesia selamat virus corona pkb depo...,yes


In [9]:
X_train = train["text_a"]
X_test = test["text_a"]
y_train = train["label"]
y_test = test["label"]

### Data preprocessing

In [10]:
X_train = X_train.apply(lambda x: ' '.join(simple_preprocess(x)))

### Tokenizing

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def tokenize(x) -> dict:
    output = tokenizer(
        text=list(x),
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="tf",
    )

    return {"input_ids": output["input_ids"], "attention_mask": output["attention_mask"], "token_type_ids": output["token_type_ids"]}

In [13]:
encoder = LabelEncoder()

X_train = tokenize(X_train)
y_train = encoder.fit_transform(y_train)
X_test = tokenize(X_test)
y_test = encoder.fit_transform(y_test)

## Persiapan model BERT

In [14]:
bert_model = TFAutoModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
def create_model():
    bert_layer = bert_model.bert

    input_ids = tf.keras.layers.Input(shape=(512,), dtype="int32", name="input_ids")
    token_type_ids = tf.keras.layers.Input(shape=(512,), dtype="int32", name="token_type_ids")
    attention_mask = tf.keras.layers.Input(shape=(512,), dtype="int32", name="attention_mask")

    layer = bert_layer([input_ids, token_type_ids, attention_mask])[0]
    layer = tf.keras.layers.LSTM(128)(layer)
    layer = tf.keras.layers.Dropout(rate=.1)(layer)
    layer = tf.keras.layers.Dense(1, activation="sigmoid")(layer)

    model = tf.keras.Model(inputs = {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
    }, outputs=layer)
    
    return model

In [16]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'token_type_ids[0][0]',     

## Learning

In [17]:
history = model.fit(x = X_train, 
                    y = y_train,
                    validation_data=(X_test, y_test),
                    epochs=2,
                    batch_size=8,
                    verbose=True)

Epoch 1/2


2701/2701 [==============================] - 2999s 1s/step - loss: 0.4444 - binary_accuracy: 0.8008 - val_loss: 0.3691 - val_binary_accuracy: 0.8275
Epoch 2/2
2701/2701 [==============================] - 2976s 1s/step - loss: 0.2987 - binary_accuracy: 0.8791 - val_loss: 0.4213 - val_binary_accuracy: 0.8289


In [18]:
y_pred = model.predict(X_test)

## Hasil

In [19]:
print(classification_report(np.round(y_pred), y_test))

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88      1890
         1.0       0.80      0.63      0.70       910

    accuracy                           0.83      2800
   macro avg       0.82      0.78      0.79      2800
weighted avg       0.83      0.83      0.82      2800



In [20]:
hasil = ['yes' if np.round(pred) == 1 else 'no' for pred in y_pred]
test['Hasil'] = hasil

In [21]:
# export result to CSV

In [22]:
test.to_csv("learning_result.csv", encoding='utf-8', index=False)